<p align="center"><img width="50%" src="https://aimodelsharecontent.s3.amazonaws.com/aimodshare_banner.jpg" /></p>


---


<p align="center"><h1 align="center">Fashion-MNIST Image Classification Model Submission Guide - Predictions Only (No Model Metadata Extraction)

---
Let's share our models to a centralized leaderboard, so that we can collaborate and learn from the model experimentation process...

**Instructions:**
1.   Get data in and set up X_train / X_test / y_train
2.   Preprocess data
3. Fit model on preprocessed data
4. Generate predictions from X_test data and submit to competition
5. Repeat submission process to improve place on leaderboard



## 1. Get data in and set up X_train, X_test, y_train objects

In [ ]:
#install aimodelshare library
! pip install aimodelshare --upgrade

In [2]:
# Get competition data
from aimodelshare import download_data
download_data('public.ecr.aws/y2e2a1d6/fashion_mnist_competition_data-repository:latest') 


Data downloaded successfully.


##2.   Preprocess data


In [3]:
# Here is a pre-designed preprocessor, but you could also build your own to prepare the data differently

def preprocessor(image_filepath, shape=(28, 28)):
        """
        This function reads in images, resizes them to a fixed shape and
        min/max transforms them before converting feature values to float32 numeric values
        required by onnx files.
        
        params:
            image_filepath
                full filepath of a particular image
                      
        returns:
            X
                numpy array of preprocessed image data
        """
           
        import cv2
        import numpy as np

        "Resize a color image and min/max transform the image"
        img = cv2.imread(image_filepath) # Read in image from filepath.
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # cv2 reads in images in order of blue green and red, we reverse the order for ML.
        img = cv2.resize(img, shape) # Change height and width of image.
        img = img / 255.0 # Min-max transform.

        # Resize all the images...
        X = np.array(img)
        X = np.expand_dims(X, axis=0) # Expand dims to add "1" to object shape [1, h, w, channels] for keras model.
        X = np.array(X, dtype=np.float32) # Final shape for onnx runtime.
        return X

In [4]:
#  Create training data objects

# Preprocess X_train image data to generate predictions from models 
import numpy as np

file_names = [('fashion_mnist_competition_data/training_data/train_' + str(i) + '.jpeg') for i in range(60000)]
preprocessed_image_data = [preprocessor(x) for x in file_names]

#Create single X_test array from preprocessed images
X_train = np.vstack(preprocessed_image_data) 

# Load y_train labels 
import pickle
with open("fashion_mnist_competition_data/y_train_labels.pkl", "rb") as fp:  
    y_train_labels = pickle.load(fp)

# One-hot encode y_train labels (y_train.columns used to generate prediction labels below)
import pandas as pd
y_train = pd.get_dummies(y_train_labels)

In [5]:
# Create test data objects

# Preprocess X_test image data to generate predictions from models 
import numpy as np

file_names = [('fashion_mnist_competition_data/test_data/test_' + str(i) + '.jpeg') for i in range(10000)]
preprocessed_image_data = [preprocessor(x) for x in file_names]

#Create single X_test array from preprocessed images
X_test = np.vstack(preprocessed_image_data) 

In [6]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)

(60000, 28, 28, 3)
(10000, 28, 28, 3)
(60000, 10)


##3. Fit model on preprocessed data


In [ ]:
# Write and execute code to fit model on preprocessed data here.

In [7]:
#Set credentials using modelshare.org username/password

from aimodelshare.aws import set_credentials
    
apiurl="https://nt9u2xcbdf.execute-api.us-east-1.amazonaws.com/prod/m"
#This is the unique rest api that powers this Fashhion-MNIST Image Classification Playground

set_credentials(apiurl=apiurl)

AI Modelshare Username:··········
AI Modelshare Password:··········
AI Model Share login credentials set successfully.


In [8]:
#Instantiate Competition
import aimodelshare as ai
mycompetition= ai.Competition(apiurl)

## **4. Submit Model predictions to leaderboard (without extracting model architecture information):**
- model metadata extraction allows you use compare_models() and instantiate_model() functions.

In [9]:
#Generate a list of predictions using X_test data

# This example uses randomly chosen values from y_train to generate a list of predictions
predicted_values = list(y_train_labels.sample(n=len(X_test)))

In [ ]:
#Submit Model predictions to leaderboard (without extracting model architecture information): 

# Submit to Competition Leaderboard
mycompetition.submit_model(model_filepath = None,
                                 preprocessor_filepath=None,
                                 prediction_submission=predicted_values)

In [11]:
# Get leaderboard to explore current best model architectures

# Get raw data in pandas data frame
data = mycompetition.get_leaderboard()

# Stylize leaderboard data
mycompetition.stylize_leaderboard(data)

,accuracy,f1_score,precision,recall,ml_framework,deep_learning,model_type,depth,num_params,conv2d_layers,maxpooling2d_layers,flatten_layers,dropout_layers,dense_layers,relu_act,softmax_act,loss,optimizer,memory_size,username,version
0,87.98%,87.84%,87.94%,87.98%,keras,True,Sequential,11.000000,88688.000000,4.000000,2.000000,1.000000,2.000000,2.000000,5.000000,1.000000,str,RMSprop,356520.000000,IEOR_Data_Mining,3
1,85.36%,85.37%,85.53%,85.36%,keras,True,Sequential,11.000000,47168.000000,4.000000,2.000000,1.000000,2.000000,2.000000,5.000000,1.000000,str,RMSprop,190440.000000,IEOR_Data_Mining,1
2,82.62%,83.03%,84.80%,82.62%,keras,True,Sequential,5.000000,514698.000000,nan,nan,1.000000,nan,4.000000,3.000000,1.000000,str,RMSprop,2059888.000000,IEOR_Data_Mining,2
3,10.92%,10.90%,10.89%,10.92%,unknown,nan,unknown,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,IEOR_Data_Mining,4


You can also compare two or more models for any models submitted to the leaderboard using example code for model metadata extraction (see code tab for this competition at www.modelshare.org for submission examples.)
```
data=mycompetition.compare_models([1,2], verbose=1)
mycompetition.stylize_compare(data)
```



#####  (Optional Extension) Submit Model With Custom Metadata: 
Can use to add team names or any other missing data you may wish to share on the leaderboard


In [ ]:
# Custom metadata can be added by passing a dict to the custom_metadata argument of the submit_model() method
# This option can be used to fill in missing data points or add new columns to the leaderboard

custom_meta = {'team': 'team one',
               'model_type': 'your_model_type',
               'new_column': 'new metadata'}

mycompetition.submit_model(model_filepath = None,
                                 preprocessor_filepath=None,
                                 prediction_submission=predicted_values,
                                 custom_metadata = custom_meta)